# Pratik Kadam

#### Problem Statement: 
A famous 30 year old pizza brand which has outlets in more than 90 countries started home
delivery services a couple of years ago and the business has grown much faster than expected.
However, outlet vendors are very much disappointed with few customers for their cheating
activities. This is because vendors, shockingly, came to know that few customers after receiving the
delivery are raising tickets for refund in the name of burnt pizzas. Even though customers received
a good pizza but still few customers are trying to cheat vendors. To overcome this issue, Franchise
has come up with an idea to integrate a pizza detection model in their application where customers
can upload images for the burnt pizzas delivered. For example, if I have received a burnt pizza then
I can upload a couple of images of the pizza to the application and it will classify the pizza as burnt
or good in order to process my refund ticket.

#### Goal:
To build a model where it accepts the images of pizza and detects as burnt pizza or good pizza.

### Importing libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from matplotlib.image import imread

### Data importing

In [ ]:
# Define the paths to the training data
train_dir = r"D:\PGA WE DL EXAM\PGA Wkend DL Exam\Datasets\ANN\Pizza Refund\train"

burnt_pizza_dir = os.path.join(train_dir, 'Burnt_pizza')
good_pizza_dir = os.path.join(train_dir, 'Good_pizza')

### Load and preprocess the images

In [ ]:
def load_images_from_directory(directory, label):
    images_path = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith((".jpg", ".png", ".jpeg",'.gif')):
            img = keras.preprocessing.image.load_img(os.path.join(directory, filename), target_size=(64, 64))
            img = keras.preprocessing.image.img_to_array(img)
            images_path.append(img)
            labels.append(label)
    return images_path, labels

burnt_pizza_images, burnt_pizza_labels = load_images_from_directory(burnt_pizza_dir, 0)
good_pizza_images, good_pizza_labels = load_images_from_directory(good_pizza_dir, 1)

In [ ]:
# Combine burnt and good pizza data
X_train = np.array(burnt_pizza_images + good_pizza_images)
y_train = np.array(burnt_pizza_labels + good_pizza_labels)

# Flatten the images
X_train = X_train.reshape(X_train.shape[0], -1)

# Normalize the pixel values to a common scale (0-1)
X_train = X_train / 255.0


### Model

In [ ]:
model = keras.Sequential([
    # Input layer
    keras.layers.Input(shape=(64*64*3,)),
    
    # Hidden layers
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    
    # Output layer
    keras.layers.Dense(1, activation='sigmoid')
])


### Compile the Model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

### Model Evaluation

In [ ]:
# Calculate and print train accuracy
train_predictions = model.predict(X_train)
train_labels = (train_predictions > 0.5).astype(int)
train_accuracy = accuracy_score(y_train, train_labels)
print(f"Train Accuracy: {train_accuracy}")


### Test Data Loading

In [ ]:
# Define the path to the directory containing test images
test_dir = r"D:\PGA WE DL EXAM\PGA Wkend DL Exam\Datasets\ANN\Pizza Refund\test"

In [ ]:
# Load and preprocess the test images
def load_test_images_from_directory(directory):
    test_images = []
    test_file_names = []
    
    for filename in os.listdir(directory):
        if filename.endswith((".jpg", ".png", ".jpeg",'.gif')):
            img = keras.preprocessing.image.load_img(os.path.join(directory, filename), target_size=(64, 64))
            img = keras.preprocessing.image.img_to_array(img)
            test_images.append(img)
            test_file_names.append(filename)
    return test_images, test_file_names

test_images, test_file_names = load_test_images_from_directory(test_dir)

In [ ]:
# Convert the test images to a numpy array and flatten
X_test_new = np.array(test_images)
X_test_new = X_test_new.reshape(X_test_new.shape[0], -1)
X_test_new = X_test_new / 255.0  # Normalize the pixel values

## Predictions

In [ ]:
predictions = model.predict(X_test_new)

In [ ]:
# Convert predictions to binary labels (0 for burnt, 1 for good)
predicted_labels = (predictions > 0.5).astype(int)

In [ ]:
# Print the predicted labels for the test images and display the images

for i, label in enumerate(predicted_labels):
    image = imread(os.path.join(test_dir, test_file_names[i]))
    plt.figure()
    plt.imshow(image)
    plt.title(f"{test_file_names[i]} is {'Burnt' if label == 0 else 'Good'} pizza.")
    plt.axis('off')
    plt.show()

# END